<a href="https://colab.research.google.com/github/juyounLee/DeepLearningZeroToAll_TF2./blob/main/07_01_01_train_test_learning_rate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import math
import numpy as np

In [2]:
# training and test dataset

x_train = [[1, 2, 1],
          [1, 3, 2],
          [1, 3, 4],
          [1, 5, 5],
          [1, 7, 5],
          [1, 2, 5],
          [1, 6, 6],
          [1, 7, 7]]

y_train = [[0, 0, 1],
          [0, 0, 1],
          [0, 0, 1],
          [0, 1, 0],
          [0, 1, 0],
          [0, 1, 0],
          [1, 0, 0],
          [1, 0, 0]]

# Evaluation our model using this test dataset
x_test = [[2, 1, 1],
          [3, 1, 2],
          [3, 3, 4]]
y_test = [[0, 0, 1],
          [0, 0, 1],
          [0, 0, 1]]

In [3]:
x_train = np.array(x_train, dtype=np.float32)
y_train = np.array(y_train, dtype=np.int32)

x_test = np.array(x_test, dtype=np.float32)
y_test = np.array(y_test, dtype=np.int32)

In [4]:
dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(len(x_train))

In [5]:
# initialize the variables

W = tf.Variable(tf.random.normal([3, 3]))
b = tf.Variable(tf.random.normal([3]))

In [6]:
# logits function
def logit_fn(x):
  logit = tf.matmul(x, W) + b
  return logit

print(logit_fn(x_train).numpy())

[[-0.24827361  1.2205905  -2.0596945 ]
 [-1.0338428   2.2924223  -2.52893   ]
 [-1.5994592   2.3946981  -2.6730804 ]
 [-2.8877897   4.4872236  -3.5394764 ]
 [-3.8933115   6.52861    -4.333797  ]
 [-1.3795063   1.4251426  -2.3479955 ]
 [-3.673359    5.559055   -4.008712  ]
 [-4.458928    6.630886   -4.477947  ]]


In [7]:
# hypothesis function
def hypothesis_fn(x):
  logit = logit_fn(x)
  hypothesis = tf.nn.softmax(logit)
  return hypothesis

In [8]:
# cost function
def cost_fn(x, y):
  logit = logit_fn(x)
  hypothesis = hypothesis_fn(x)
  cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(y, logits=logit))
  return cost

In [9]:
# gradient function
def grad(x, y):
  with tf.GradientTape() as tape:
    cost = cost_fn(x, y)
  return tape.gradient(cost, [W, b])

optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)

In [10]:
def pred_fn(x, y):
  y_pred = tf.argmax(hypothesis_fn(x), 1)
  correct_value = tf.math.equal(y_pred, tf.argmax(y, 1))
  accuracy = tf.reduce_mean(tf.cast(correct_value, dtype=tf.float32))
  return accuracy

In [11]:
for step in range(2001):
  for feature, label in iter(dataset):
    grads = grad(feature, label)
    optimizer.apply_gradients(grads_and_vars=zip(grads, [W, b]))
    cost = cost_fn(feature, label)

    if step%200 == 0:
      print('step: {} cost: {:.4f}'.format(step, cost.numpy()))

step: 0 cost: 4.0389
step: 200 cost: 1.2280
step: 400 cost: 1.0223
step: 600 cost: 0.9023
step: 800 cost: 0.8255
step: 1000 cost: 0.7716
step: 1200 cost: 0.7310
step: 1400 cost: 0.6989
step: 1600 cost: 0.6726
step: 1800 cost: 0.6504
step: 2000 cost: 0.6313


In [12]:
# calculate the accuracy
accuracy = pred_fn(x_test, y_test)
print('Accuracy: {:.5f}'.format(accuracy))

Accuracy: 1.00000


In [13]:
x = [[3, 3, 4]]
x = np.array(x, dtype=np.float32)
print(logit_fn(x))
y_pred = hypothesis_fn(x)
print(tf.argmax(y_pred, 1))

tf.Tensor([[-6.3982244  -0.47135746  2.1973803 ]], shape=(1, 3), dtype=float32)
tf.Tensor([2], shape=(1,), dtype=int64)
